# RNN을 이용한 텍스트 생성

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
seed=2022
np.random.seed(seed)
tf.random.set_seed(seed)

In [2]:
text = '''경마장에 있는 말이 뛰고 있다.
그의 말이 법이다.
가는 말이 고와야 오는 말이 곱다.'''

### 텍스트 전처리

In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [4]:
# 단어 집합 생성
t = Tokenizer()
t.fit_on_texts([text])

In [5]:
# 특징
# 1. 가나다 순이 아닌 많이 나온 단어 순서
# 2. 1에서부터 시작(0부터 시작하지 않음)
t.word_index

{'가는': 8,
 '경마장에': 2,
 '고와야': 9,
 '곱다': 11,
 '그의': 6,
 '뛰고': 4,
 '말이': 1,
 '법이다': 7,
 '오는': 10,
 '있는': 3,
 '있다': 5}

In [7]:
# 단어 집합 크기 - keras tokenizer의 시작 인덱스가 1이기 때문에 1을 더해준다
vocab_size = len(t.word_index)+1
vocab_size

12

In [9]:
t.texts_to_sequences(['그의 말이 법이다.'])

[[6, 1, 7]]

In [10]:
# corps를 encoding한 결과
for line in text.split('\n'):
    encoded=t.texts_to_sequences([line])[0]
    print(encoded)

[2, 3, 1, 4, 5]
[6, 1, 7]
[8, 1, 9, 10, 1, 11]


In [11]:
t.texts_to_sequences([text])

[[2, 3, 1, 4, 5, 6, 1, 7, 8, 1, 9, 10, 1, 11]]

 - sequence 생성

In [13]:
sq=[]
for line in text.split('\n'):
    encoded=t.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sq.append(encoded[: i+1])

sq

[[2, 3],
 [2, 3, 1],
 [2, 3, 1, 4],
 [2, 3, 1, 4, 5],
 [6, 1],
 [6, 1, 7],
 [8, 1],
 [8, 1, 9],
 [8, 1, 9, 10],
 [8, 1, 9, 10, 1],
 [8, 1, 9, 10, 1, 11]]

In [16]:
# sequence 중에서 제일 길이가 긴 거
max_len = len(max(sq))
max_len

6

 - padding
    - 전체 샘플 길이를 max_len에 맞추도록 0 추가
    - 앞쪽을 채우는 pre 옵션이 디폴트

In [17]:
sequences = pad_sequences(sq, maxlen=max_len, padding='pre')
sequences[:5]

array([[0, 0, 0, 0, 2, 3],
       [0, 0, 0, 2, 3, 1],
       [0, 0, 2, 3, 1, 4],
       [0, 2, 3, 1, 4, 5],
       [0, 0, 0, 0, 6, 1]], dtype=int32)

- X, y 데이터 설정

In [18]:
X = sequences[:,:-1]
y = sequences[:,-1]
X.shape, y.shape

((11, 5), (11,))

In [19]:
Y = to_categorical(y)
Y.shape

(11, 12)

### 모델
 - Embedding
 - SimpleRNN
 - Dense

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

In [22]:
# Embedding =4, SimpleRNN = 32
# Embedding vector 개수 = RNN layer에서 feature 개수
model = Sequential([
                    Embedding(vocab_size, 4, input_length=max_len-1),
                    SimpleRNN(32), # activation = 'tanh'는 디폴트 옵션
                    Dense(vocab_size, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 5, 4)              48        
                                                                 
 simple_rnn (SimpleRNN)      (None, 32)                1184      
                                                                 
 dense (Dense)               (None, 12)                396       
                                                                 
Total params: 1,628
Trainable params: 1,628
Non-trainable params: 0
_________________________________________________________________


 - 모델 학습

In [23]:
model.compile('adam', 'categorical_crossentropy', ['accuracy'])

In [24]:
hist = model.fit(X, Y, epochs=200, verbose=0)

In [25]:
hist.history['accuracy'][-1] #model.evaluate(X, Y)랑 같음

1.0

- 모델 검증

In [26]:
from google.colab import files
up = files.upload()

Saving my_util.py to my_util.py


In [27]:
from my_util import sentence_generation
sentence_generation(model, t, max_len, '경마장에', 3)

'경마장에 있는 말이 뛰고'

In [28]:
sentence_generation(model, t, max_len, '가는', 5)

'가는 말이 고와야 오는 말이 곱다'

### 모델 변화
 - Embedding vector 갯수 [2,4,6]
 - RNN 유닛 개수: [24, 36, 48]

In [31]:
for n_embed in [2,4,6]:
    for n_unit in [24,32,48]:
        model = Sequential([
                    Embedding(vocab_size, n_embed, input_length=max_len-1),
                    SimpleRNN(n_unit), # activation = 'tanh'는 디폴트 옵션
                    Dense(vocab_size, activation='softmax')
        ])
        model.compile('adam','categorical_crossentropy', ['accuracy'])
        hist = model.fit(X,Y,epochs=200, verbose=0)
        print('==================')
        print(f'Embedding Vector:{n_embed}, RNN_unit:{n_unit}')
        print(f"정확도:{hist.history['accuracy'][-1]:.4f}")
        print(sentence_generation(model, t, max_len, '경마장에', 4))
        print(sentence_generation(model, t, max_len, '그의', 2))
        print(sentence_generation(model, t, max_len, '가는', 5))
        print()

Embedding Vector:2, RNN_unit:24
정확도:0.8182
경마장에 말이 말이 오는 말이
그의 말이 법이다
가는 말이 법이다 오는 말이 곱다

Embedding Vector:2, RNN_unit:32
정확도:0.8182
경마장에 말이 법이다 오는 말이
그의 말이 고와야
가는 말이 고와야 오는 말이 곱다

Embedding Vector:2, RNN_unit:48
정확도:0.9091
경마장에 말이 고와야 오는 말이
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

Embedding Vector:4, RNN_unit:24
정확도:0.8182
경마장에 말이 말이 오는 말이
그의 말이 법이다
가는 말이 법이다 오는 말이 곱다

Embedding Vector:4, RNN_unit:32
정확도:1.0000
경마장에 있는 말이 뛰고 있다
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

Embedding Vector:4, RNN_unit:48
정확도:0.9091
경마장에 말이 고와야 오는 말이
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

Embedding Vector:6, RNN_unit:24
정확도:1.0000
경마장에 있는 말이 뛰고 있다
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

Embedding Vector:6, RNN_unit:32
정확도:1.0000
경마장에 있는 말이 뛰고 있다
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

Embedding Vector:6, RNN_unit:48
정확도:1.0000
경마장에 있는 말이 뛰고 있다
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

